In [172]:
import os

In [173]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [174]:
if not os.path.exists('/content/drive/MyDrive/BERT-Preguntas-Respuestas-Posgrados'):
  os.mkdir('/content/drive/MyDrive/BERT-Preguntas-Respuestas-Posgrados')

In [175]:
import json
import torch
import requests
from tqdm import tqdm

In [176]:
#Cargar los datos desde Github
url1 = 'https://raw.githubusercontent.com/Leo646/TrabajoTitulacion/main/Cojunto_de_datos/Entrenamiento.json'
url2 = 'https://raw.githubusercontent.com/Leo646/TrabajoTitulacion/main/Cojunto_de_datos/Validacion.json'

In [177]:
def leer_datos(ruta):  
  # Carga el archivo  json y almacenar 
  #el contexto, las preguntas y respuestas
  #en arreglos
  resp = requests.get(ruta)
  data = json.loads(resp.text)
  contexto = []
  preguntas = []
  respuestas= []
  for group in data['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexto.append(context)
          preguntas.append(question)
          respuestas.append(answer)

  return contexto, preguntas, respuestas

In [178]:
train_contextos, train_preguntas, train_respuestas = leer_datos(url1)
valid_contextos, valid_preguntas, valid_respuestas = leer_datos(url2)

In [179]:
import pandas as pd
user_list = list(zip(train_contextos, train_preguntas, train_respuestas))
df = pd.DataFrame( user_list,columns=['contexto','pregunta', 'respesta'])
df[:110]

,contexto,pregunta,respesta
0,Los postulantes pueden solicitar la inscripció...,¿En dónde debo inscribirme para un posgrado?,{'text': 'en el Sistema de Gestión de Inscripc...
1,Los postulantes pueden solicitar la inscripció...,¿En dónde debo inscribirme para una maestría?,{'text': 'en el Sistema de Gestión de Inscripc...
2,Los postulantes pueden solicitar la inscripció...,¿Dónde debo realizar la solicitud para un posg...,{'text': 'en el Sistema de Gestión de Inscripc...
3,Los postulantes pueden solicitar la inscripció...,¿Existen descuentos para matricularse en las m...,{'text': 'Si desea conocer información más det...
4,Los postulantes pueden solicitar la inscripció...,¿Que tipo de descuento hay para un posgrado?,{'text': 'Si desea conocer información más det...
...,...,...,...
105,Los estudiantes de posgrado que tomen asignatu...,¿Dónde debo entregar el comprobante de pago?,"{'text': 'en la Secretaría del programa', 'ans..."
106,Los estudiantes de posgrado que tomen asignatu...,¿En dónde se entrega el comprobante de pago?,"{'text': 'en la Secretaría del programa', 'ans..."
107,Los estudiantes de posgrado que tomen asignatu...,¿Cómo se calcula el pago para una segunda matr...,{'text': 'En caso de segunda y tercera matrícu...
108,Los estudiantes de posgrado que tomen asignatu...,¿Cómo se calcula el pago para una tercera matr...,{'text': 'En caso de segunda y tercera matrícu...


In [180]:

print(f'hay {len(train_preguntas)} preguntas')

print(train_preguntas[:1])
print(train_respuestas[:1])



hay 139 preguntas
['¿En dónde debo inscribirme para un posgrado?']
[{'text': 'en el Sistema de Gestión de Inscripciones (SGI)', 'answer_start': 179}]


In [181]:
def posicion_final(respuestas,contextos):
  for respuesta, contexto in zip(respuestas, contextos):
    texto_respueta = respuesta['text']  #se extrae la respuesta
    inicio_respuesta = respuesta['answer_start'] # se extrae la posición de inicio de la respuesta
    final_respuesta = inicio_respuesta+ len(texto_respueta)# se calcula la posición final de la respuesta

    
    #En ocaciones las respuestas del conjunto de datos  están mal por uno o dos caracteres, 
    #así que se debe arreglar eso
    if contexto[inicio_respuesta:final_respuesta ] == texto_respueta:
      respuesta['answer_end'] = final_respuesta 
    elif contexto[inicio_respuesta-1:final_respuesta -1] == texto_respueta:
      respuesta['answer_start'] = inicio_respuesta  - 1
      respuesta['answer_end'] = final_respuesta  - 1     
    elif contexto[inicio_respuesta-2:final_respuesta -2] == texto_respueta:
      respuesta['answer_start'] = inicio_respuesta  - 2
      respuesta['answer_end'] = final_respuesta  - 2     
posicion_final(train_respuestas,train_contextos,)
posicion_final(valid_respuestas,valid_contextos)

In [182]:
user_list = list(zip(train_contextos, train_preguntas, train_respuestas))
df = pd.DataFrame( user_list,columns=['contexto','pregunta', 'respesta'])
df[:200]

,contexto,pregunta,respesta
0,Los postulantes pueden solicitar la inscripció...,¿En dónde debo inscribirme para un posgrado?,{'text': 'en el Sistema de Gestión de Inscripc...
1,Los postulantes pueden solicitar la inscripció...,¿En dónde debo inscribirme para una maestría?,{'text': 'en el Sistema de Gestión de Inscripc...
2,Los postulantes pueden solicitar la inscripció...,¿Dónde debo realizar la solicitud para un posg...,{'text': 'en el Sistema de Gestión de Inscripc...
3,Los postulantes pueden solicitar la inscripció...,¿Existen descuentos para matricularse en las m...,{'text': 'Si desea conocer información más det...
4,Los postulantes pueden solicitar la inscripció...,¿Que tipo de descuento hay para un posgrado?,{'text': 'Si desea conocer información más det...
...,...,...,...
134,Las programas realizarán la homologación media...,¿Qué es la validación de conocimientos?,{'text': 'consiste en validar de los conocimie...
135,"Para consultar las fechas de inscripción, matr...",¿En qué fecha inician las clases?,{'text': 'Para consultar las fechas de inscrip...
136,"Para consultar las fechas de inscripción, matr...",¿Cuál es el calendario académico de las maestr...,{'text': 'Para consultar las fechas de inscrip...
137,"Para consultar las fechas de inscripción, matr...",¿Cuál es el calendario académico de los posgra...,{'text': 'Para consultar las fechas de inscrip...


In [183]:
# You can see that now we get the answer_end also
print(train_preguntas[:1],"\n")
print(train_respuestas[:1])

['¿En dónde debo inscribirme para un posgrado?'] 

[{'text': 'en el Sistema de Gestión de Inscripciones (SGI)', 'answer_start': 179, 'answer_end': 226}]


In [184]:
 ! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [185]:
from transformers import BertTokenizerFast
#tokenizar los datos en vectores numéricos
tokenizador = BertTokenizerFast.from_pretrained('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',do_lower_case=False)
#agregar tokens que el modelo no contempla en su vocabulario 
tokenizador.add_tokens(['Decano/a','8.99','7.99','50%','70%','30%','40%','60%','75%','Software','%','75','y/o','/','(SGI)',';','70','7','https://unl.edu.ec/posgrados',' https://unl.edu.ec/calendario-maestrias'])
#tokenizar el contexo y las preguntas 
train_codificacion = tokenizador(train_contextos, train_preguntas, truncation=True, padding=True)
valid_codificacion= tokenizador(valid_contextos, valid_preguntas, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [186]:
train_codificacion.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [187]:
no_of_encodings = len(train_codificacion['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 139 context-question pairs


In [188]:
train_codificacion['input_ids'][:109]
# print("")
# print("")
tokenizador.decode(train_codificacion['input_ids'][1])

'[CLS] Los postulantes pueden solicitar la inscripción a los programas de posgrado ( Maestría académica con trayectoria profesional y Maestría académica con trayectoria de investigación ) en el Sistema de Gestión de Inscripciones (SGI). Si desea conocer información más detallada sobre los posgrados con respecto a costos, duración, descuentos, entre otras cuestiones, visite el siguiente enlace : https://unl.edu.ec/posgrados [SEP] ¿ En dónde debo inscribirme para una maestría? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [189]:
tokenizador.decode(train_codificacion['input_ids'][1])

'[CLS] Los postulantes pueden solicitar la inscripción a los programas de posgrado ( Maestría académica con trayectoria profesional y Maestría académica con trayectoria de investigación ) en el Sistema de Gestión de Inscripciones (SGI). Si desea conocer información más detallada sobre los posgrados con respecto a costos, duración, descuentos, entre otras cuestiones, visite el siguiente enlace : https://unl.edu.ec/posgrados [SEP] ¿ En dónde debo inscribirme para una maestría? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [190]:
def posicion_tokens(encodings, answers):
  posicion_inicial = []
  posicion_final = []
  #se transforma la posicion inicial y final
  #de la respuetas en tokens
  for i in range(len(answers)):
    #se transforma la posición de inicio de cada respuesta en tokens
    posicion_inicial.append(encodings.char_to_token(i, answers[i]['answer_start']))
    #se transforma la posición de donde termina cada respuesta en tokens
    posicion_final.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

    if posicion_inicial[-1] is None: # si la posiscion de inicio de la respuesta es nula
      posicion_inicial[-1] = tokenizador.model_max_length
    if posicion_final[-1] is None: # si la posiscion final de la respuesta es nula
      posicion_final[-1] = tokenizador.model_max_length

  encodings.update({'start_positions': posicion_inicial, 'end_positions': posicion_final})

posicion_tokens(train_codificacion, train_respuestas)
posicion_tokens(valid_codificacion, valid_respuestas)

In [191]:
#train_codificacion['end_positions'][:150]
valid_codificacion['start_positions'][:30]

[16,
 16,
 55,
 55,
 77,
 93,
 12,
 59,
 30,
 30,
 30,
 63,
 63,
 63,
 63,
 63,
 109,
 109,
 109,
 109,
 150,
 150,
 150,
 150,
 39,
 22,
 22,
 15]

In [192]:
from torch.utils.data import Dataset
class conjunto_Datos(Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [285]:
train_dataset = conjunto_Datos(train_codificacion)
valid_dataset = conjunto_Datos(valid_codificacion)

In [383]:
#parámetros
batch_size=7
N_EPOCHS =8
lr=5e-5

In [384]:
from torch.utils.data import DataLoader

# Define los cargadores para el entrenamiento
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size)

In [385]:
from transformers import AutoModelForQuestionAnswering
#Descarga del modelo para el entrenamiento
modelo = AutoModelForQuestionAnswering.from_pretrained('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es')
modelo.resize_token_embeddings(len(tokenizador)) 


Embedding(31022, 768)

In [386]:
#Detemita si hay una GPU diposnible para el entrenamiento caso contrario se usa una CPU
dispositivo = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'trabajando con  {dispositivo}')



trabajando con  cuda


In [387]:
from transformers import AdamW
optimizador = AdamW(modelo.parameters(), lr)
modelo.to(dispositivo)#el modleo trabaja con GPU Tesla T4
modelo.train() #se entrena el modelo
for iteracion in range(N_EPOCHS):#catida de iteraciones
#permite vizualizar el progreso del entrenamiento en cada iteracion
  progreso = tqdm(train_loader, leave=True) 
  for batch in progreso: #por cada iteración el modelo recibe una cantidad de datos 
    optimizador.zero_grad()#permite mejorar el rendimiento
    #se pide lo datos para el entrenamiento 
    entrada = batch['input_ids'].to(dispositivo)
    atencion= batch['attention_mask'].to(dispositivo)
    posicion_inicial = batch['start_positions'].to(dispositivo)
    posiscion_final = batch['end_positions'].to(dispositivo)
    #se envia los datos al modelo para que genere una salida
    salida = modelo(entrada, attention_mask=atencion, start_positions=posicion_inicial, end_positions=posiscion_final)
    perdida = salida[0]#el modelo genera una pérdida por cada iteración
    perdida.backward()
    optimizador.step()
    #se imprime el progreso del entrenamiento en cada iteracion icluida la pérdida
    progreso.set_description(f'Epoch {iteracion+1}')
    progreso.set_postfix(loss=perdida.item())

Epoch 8: 100%|██████████| 20/20 [00:07<00:00,  2.63it/s, loss=0.0161]


In [388]:
model_path = '/content/drive/MyDrive/BERT-Preguntas-Respuestas-Posgrados'
modelo.save_pretrained(model_path)
tokenizador.save_pretrained(model_path)

('/content/drive/MyDrive/BERT-Preguntas-Respuestas-Posgrados/tokenizer_config.json',
 '/content/drive/MyDrive/BERT-Preguntas-Respuestas-Posgrados/special_tokens_map.json',
 '/content/drive/MyDrive/BERT-Preguntas-Respuestas-Posgrados/vocab.txt',
 '/content/drive/MyDrive/BERT-Preguntas-Respuestas-Posgrados/added_tokens.json',
 '/content/drive/MyDrive/BERT-Preguntas-Respuestas-Posgrados/tokenizer.json')

In [389]:
from transformers import BertForQuestionAnswering, BertTokenizerFast

model_path = '/content/drive/MyDrive/BERT-Preguntas-Respuestas-Posgrados'
model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'trabajado con {device}')

model = model.to(device)

trabajado con cuda


In [390]:
!nvidia-smi

Mon Sep 19 20:23:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    31W /  70W |   9726MiB / 15109MiB |     16%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [391]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [392]:
model.eval()

acc = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

acc = sum(acc)/len(acc)
print("\n",acc)
print("\n\nT/P\tanswer_start\tanswer_end\n")
for i in range(len(start_true)):
  print(f"true\t{start_true[i]}\t{end_true[i]}\n"
        f"pred\t{start_pred[i]}\t{end_pred[i]}\n")

100%|██████████| 4/4 [00:00<00:00,  9.72it/s]


 0.7857142984867096


T/P	answer_start	answer_end

true	150	187
pred	150	187

true	150	187
pred	150	187

true	150	187
pred	150	187

true	39	42
pred	20	42

true	22	25
pred	8	27

true	22	25
pred	8	27

true	15	32
pred	15	32



In [393]:
def prediccion(contexto, pregunta):
  #se toqueniza las preguntas y el contexto
  entradas = tokenizer.encode_plus(pregunta, contexto, return_tensors='pt').to(device)
  salida = model(**entradas)
  inicio_respuesta = torch.argmax(salida[0])  
  final_respuesta = torch.argmax(salida[1]) + 1 
  respuesta = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(entradas['input_ids'][0][inicio_respuesta:final_respuesta]))
  return respuesta

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re
  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)
  def white_space_fix(text):
    return " ".join(text.split())

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(lower(s)))
 
def exact_match(prediction, truth):
    return bool(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  

  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  common_tokens = set(pred_tokens) & set(truth_tokens)

  if len(common_tokens) == 0:
    return 0
  
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  
  return round(2 * (prec * rec) / (prec + rec), 2)
  
def question_answer(context, question,answer):
  prediction = prediccion(context,question)
  em_score = exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f'Pregunta: {question}')
  print(f'Prediccion: {prediction}')
  print(f'Respuesta verdadeda: {answer}')
  print(f'Coindencia exacta: {em_score}')
  print(f'F1 Presicion: {f1_score}\n')

In [395]:
context = """Para la homologación mediante análisis comparativo de conocimientos, se requieren los siguientes requisitos: El aspirante realizará la solicitud de homologación al Decano de la Facultad, adjuntando lo siguiente: Récord académico con las calificaciones de las asignaturas, certificado de no haber agotado la tercera matrícula en la carrera de destino, programas analíticos y sílabos aprobados por la institución de procedencia. Cuando la homologación se realice por validación de conocimientos, en la planificación de la prueba se considerarán los siguientes componentes: información general, antecedentes; nombre de las asignaturas a validar, estructura del examen de validación, responsable de la elaboración y aplicación del examen, información del postulante, plataforma a utilizar para la aplicación del examen en forma virtual o presencial, cronograma de ejecución y firmas de responsabilidad"""
questions = [
           "validación de conocimientos?",
         

             ]

answers = [" Cuando la homologación se realice por validación de conocimientos, en la planificación de la prueba se considerarán los siguientes componentes : información general, antecedentes ; nombre de las asignaturas a validar, estructura del examen de validación, responsable de la elaboración y aplicación del examen, información del postulante, plataforma a utilizar para la aplicación del examen en forma virtual o presencial, cronograma de ejecución y firmas de responsabilidad"]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

Pregunta: validación de conocimientos?
Prediccion: Cuando la homologación se realice por validación de conocimientos, en la planificación de la prueba se considerarán los siguientes componentes : información general, antecedentes ; nombre de las asignaturas a validar, estructura del examen de validación, responsable de la elaboración y aplicación del examen, información del postulante, plataforma a utilizar para la aplicación del examen en forma virtual o presencial, cronograma de ejecución y firmas de responsabilidad
Respuesta verdadeda:  Cuando la homologación se realice por validación de conocimientos, en la planificación de la prueba se considerarán los siguientes componentes : información general, antecedentes ; nombre de las asignaturas a validar, estructura del examen de validación, responsable de la elaboración y aplicación del examen, información del postulante, plataforma a utilizar para la aplicación del examen en forma virtual o presencial, cronograma de ejecución y firmas 